### Publicis Media: Composition predictors

##### Data

In [17]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/emolodniak/jupyter/main/randomforest/displaytest.csv')
df.head()

,TOUCHPOINT,MONTH,SITE,IMPRESSIONS,AGE_MIN,AGE_DIFF,SEX,INCOME,UNWEIGHTED_REACH,COMPOSITION
0,DISPLAY,2018-01-01,VK,10095780,16,28,W,1,435,0.647
1,DISPLAY,2016-02-01,MAILPOST,124038798,25,30,P,1,3495,0.880
2,IVIDEO,2016-08-01,IMHO,1549885,20,34,W,1,89,0.597
3,DISPLAY,2016-03-01,OK,22885630,20,15,W,1,367,0.636
4,IVIDEO,2016-06-01,GPMD,781124,16,19,W,0,91,0.766


##### Data info

In [21]:
pd.set_option('display.max_colwidth', None)
dic = pd.read_csv('https://raw.githubusercontent.com/emolodniak/jupyter/main/randomforest/dictionary.csv')
dic

,Variable,Value,Format,Comment
0,TOUCHPOINT,DISPLAY/IVIDEO,Text,-
1,MONTH,Period,Date,dd.mm.yyyy
2,SITE,Site name,Text,-
3,IMPRESSIONS,Impressions,Number,-
4,AGE_MIN,Target audience minimum age,Number,-
5,AGE_DIFF,Target audience age difference,Number,(Target audience maximum age) - (Target audience minimum age)
6,SEX,Sex,Text,"W - woman, M - man, P - all"
7,INCOME,Income group,Binary,"0 - without income group, 1 - mid+"
8,UNWEIGHTED_REACH,Sample size,Number,Sample size in TNS panel
9,COMPOSITION,Composition,Number,-
